Microservices - REST (Synchron)
-------------------------------

![](images/Microservices-REST.png)

Quelle: Buch Microservices Rezepte
- - -

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Zusätzlich ist ein Service **Webshop** am laufen, der dem Benutzer mit einer Webseite einen einfachen Einstieg in das System ermöglicht.

- - -

Zuerst erstellen wir den Kubernetes Namespace ms-rest und Labeln diesen, dass Kiali Informationen sammeln kann.

In [ ]:
! kubectl create namespace ms-rest
! kubectl label  namespace ms-rest istio-injection=enabled

Jetzt ist ein guter Zeitpunkt um das Kubernetes Dashboard zu starten und dort den Namespace "ms-rest" auszuwählen.

Dann sehen wir, wie beim nächsten Schritt, die Microservices erstellt werden.

In [ ]:
%%bash
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/catalog-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/customer-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/order-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/catalog-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/customer-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/order-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/webshop-service.yaml
kubectl get service/webshop --namespace ms-rest

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten.

In [ ]:
! [ ! -f ~/work/server-ip ] && { echo "replace-with-server-ip" >~/work/server-ip; }
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

---
## Kiali — Observability


Kiali vom, Projekt Istio, hilft die Struktur (= Service Mesh) der Applikation und deren Microservices und Schnittstellen zu verstehen.

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

- - - 
## Jaeger — Tracing

Jaeger ist ein System, um Aufrufe zwischen Microservices zu verfolgen..

In [ ]:
! kubectl get service -n istio-system -l app=jaeger -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Jaeger  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service/tracing -o jsonpath='{.spec.ports[?(@.name=="http-query")].nodePort}')

- - -
## Lasttest

Um die Verbindungen sichtbar zu machen, erzeugen wir ein wenig Traffic.

In [ ]:
%%bash
for i in `seq 1 1000`;
do
    curl -s -o /dev/null -I -w "%{http_code}" http://order.ms-rest:8080/order
    echo
done

- - - 
### REST Paradigma

Representational State Transfer (abgekürzt REST, seltener auch ReST) bezeichnet ein Programmierparadigma für verteilte Systeme.

REST ist eine Abstraktion der Struktur und des Verhaltens des World Wide Web (HTTP GET, PUT, POST, DELETE).

REST hat das Ziel, einen Architekturstil zu schaffen, der die Anforderungen des modernen Web besser darstellt. Dabei unterscheidet sich REST vor allem in der Forderung nach einer einheitlichen Kommunikation Schnittstelle von anderen Architekturstilen.

Um das REST Paradigma zu verdeutlichen, führen wir mittels `curl` (Windows `Invoke-WebRequest`) mehrere API Aufrufe gegen unsere Microservices aus:

In [ ]:
! curl http://webshop.ms-rest:8080/webshop/customer/customer/api

In [ ]:
! curl http://webshop.ms-rest:8080/webshop/catalog/catalog/api

In [ ]:
! curl http://webshop.ms-rest:8080/webshop/order/order/api

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-rest

- - -
## Grafana

Grafana ist eine plattformübergreifende Open-Source-Anwendung zur grafischen Darstellung von Daten aus verschiedenen Datenquellen wie z. B. InfluxDB, MySQL, PostgreSQL, Prometheus und Graphite. Die erfassten Rohdaten lassen sich anschließend in verschiedenen Anzeigeformen ausgeben

In [ ]:
! kubectl get service -n istio-system -l app.kubernetes.io/instance=grafana -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Grafana  UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app.kubernetes.io/instance=grafana -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')                                                         